In [133]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
sc.stop()
sc = SparkContext('local')
spark = SparkSession(sc)

In [134]:
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
print(len(stops))
filled_p = ['ah','oh','eh','er','erm','mm','mhm','huh','uhu']
for p in filled_p:
    stops.add(p)
print(stops)
print(len(stops))

179
{'out', 'we', 'was', 'how', 'myself', 'for', 'they', 'about', "hasn't", 'huh', 'then', 'both', 'so', 're', 'don', 'm', 'as', 'any', 'mightn', 'after', 'you', 'wouldn', 'why', 'been', 'where', 'by', "isn't", 'yourself', 'wasn', 'a', 'er', "haven't", 'did', "hadn't", 'uhu', 'their', 'hasn', 'doing', 'be', 'further', 'ours', 'now', 'am', 'her', "you'll", 'yourselves', 'that', 'my', 'what', 'to', 'd', 'not', "won't", "couldn't", 'own', 'there', 'this', 'mhm', 'each', 'all', 'haven', 'more', 'me', 've', 'weren', 'which', 'himself', 'nor', 'other', "shouldn't", 'who', "should've", 'erm', 'same', 'at', 'such', 't', 'up', 'than', 'mm', 'can', "you've", 'too', 'ah', 'these', 'while', "wasn't", 'ourselves', 'before', 'i', 'he', "didn't", 'our', 'its', 'but', 'with', "wouldn't", 'those', 'because', 'the', 'y', 'shouldn', 'it', 'mustn', 'hers', 'just', 'doesn', 'ain', 'between', 'over', 'had', 'aren', "mightn't", 'does', 'have', 'and', 'or', 'some', "mustn't", 'only', 'won', 'when', 'needn', '

In [135]:
df = spark.read.csv(
    "SN2C.tsv", sep=r'\t', header=False, mode="DROPMALFORMED")

dataColl=df.collect()
for row in dataColl[0:5]:
    print(row[0] + "," +str(row[1]))

S0653,okay tell us about your dream
S0655,well it start off as me fly in a spaceship
S0653,mm
S0655,and then our ship crash for some reason and we be in the middle of a desert
S0654,who be in there as well who be in there as well


In [136]:
def extract_speakers_n(df1, n):
    dataC = df1.collect()
    speaker = [dataC[0][0]]
    for row in dataC:
        if row[0] not in speaker:
            speaker.append(row[0])
        if len(speaker) == n:
            break
    #rdd1 = rdd1.rdd
    return df1, speaker

In [137]:
def extract_dialogues_dic_n(rdd1, speaker):
    n = len(speaker)
    dataC = rdd1.collect()
    speaker_dic = [{} for x in range(0,n)]
    
    
    for i in range(0,len(dataC)):
        if dataC[i][0] in speaker:
            p = speaker.index(dataC[i][0])
            if dataC[i][1]!=None:
                speaker_dic[p][i] = dataC[i][1].split()
    
    return speaker, speaker_dic

In [138]:
def get_er_one_with_u(x,y, exclude_stops):
# need to be changed in other files
    record=[[0 for i in range(len(y)+1)]  for j in range(len(x)+1)]
    dic_record_s1 = {}
    dic_record_s2 = {}
    expression_list_1 = []
    expression_list_2 = []
    expression_list_1_token = []
    expression_list_2_token = []
    for i in range(0,len(x)):
        for j in range(0,len(y)):
            if x[i] == y[j]:
                record[i+1][j+1] = record[i][j] + 1
                if record[i+1][j+1] > 0:
                        if i not in dic_record_s1.keys():
                            dic_record_s1[i] = record[i+1][j+1]
                        elif i in dic_record_s1.keys():
                            if (dic_record_s1[i]<record[i+1][j+1]):
                                dic_record_s1[i] = record[i+1][j+1]
                                
                        if j not in dic_record_s2.keys():
                            dic_record_s2[j] = record[i+1][j+1]
                        elif j in dic_record_s2.keys():
                            if (dic_record_s2[j]<record[i+1][j+1]):
                                dic_record_s2[j] = record[i+1][j+1]

    if dic_record_s1== {}:
        return []
    
    else:
        i = max(dic_record_s1.values())
        
        
    if dic_record_s2== {}:
        return []
    
    else:
        j = max(dic_record_s2.values())
    
    while i>0:  
        for key in dic_record_s1:
            if dic_record_s1[key] == i:
            #expression_repitition_record = expression_repitition_record+1
                expression = x[key+1-i : key+1]
                expression_string = ' '.join(expression)
            
                for k in range(0,i):
                    dic_record_s1[key - k] = 0
            
                if exclude_stops == True:
                    if expression_string not in stops:
                        if expression_string not in expression_list_1:
                            expression_list_1.append(expression_string)
                            expression_list_1_token.append(expression)
                                     
                else:
                    if expression_string not in expression_list_1:
                        expression_list_1.append(expression_string)
                        expression_list_1_token.append(expression)
        
        i = i-1
        
    while j>0:  
        for key in dic_record_s2:
            if dic_record_s2[key] == j:
                expression_2 = y[key+1-j : key+1]
                expression_string_2 = ' '.join(expression_2)
            
                for k in range(0,j):
                    dic_record_s2[key - k] = 0
                
                
                
                if exclude_stops == True:
                    if expression_string_2 not in stops:
                        if expression_string_2 not in expression_list_1:
                            expression_list_1.append(expression_string_2)
                            expression_list_1_token.append(expression_2)
                
                else:
                    if expression_string_2 not in expression_list_1:
                        expression_list_1.append(expression_string_2)
                        expression_list_1_token.append(expression_2)
        
        j = j-1
        
    return expression_list_1

In [139]:
def get_er_dic(speaker1_dialogue, speaker2_dialogue):
    expressions = {}
    
    for x,value_x in speaker1_dialogue.items():
        for y,value_y in speaker2_dialogue.items():
            string = get_er_one_with_u(value_x,value_y, False)
            
            for s in string:
                
                if s not in expressions.keys():
                    
                    pos = max(x,y)
                    expressions[s] = [pos,[x],[y]]
                    
                elif s in expressions.keys():
                    if (x in expressions[s][1]) and (y not in expressions[s][2]):
                        expressions[s][2].append(y)
                        expressions[s][0] = max(min(expressions[s][2]),min(expressions[s][1]))
                        
                    elif (y in expressions[s][2]) and (x not in expressions[s][1]):
                        expressions[s][1].append(x)
                        expressions[s][0] = max(min(expressions[s][2]),min(expressions[s][1]))
    
    return expressions

In [140]:
def get_er_dic_group(speaker, speaker_dic):
    n = len(speaker)
    expression_n = []
    fit_ex_in_dic = [[]*n]
    sum_dic = {}
    
    for i in range(0,n-1):
        for j in range(i+1, n):
            ex = get_er_dic(speaker_dic[i],speaker_dic[j])
            expression_n.append(ex)
            
    for split_dic in expression_n:
        for k, v in split_dic.items():
            v_sum = []
            pos = v[0]
            
            for v1 in v[1]:
                v_sum.append(v1)
            for v2 in v[2]:
                v_sum.append(v2)
            
            if k not in sum_dic.keys():
                sum_dic[k] = [pos,v_sum]
            else:
                for vs in v_sum:
                    if vs not in sum_dic[k][1]:
                        sum_dic[k][1].append(vs)
                        sum_dic[k][0] = min(pos, sum_dic[k][0])
    
    
    for key, value in sum_dic.items():
        sum_dic[key][1] = sorted(sum_dic[key][1])
        
    return sum_dic
        

In [141]:
def get_er_window_n(speaker_dic, ex_group, u_high):
    ex_in_window = {}
    if len(speaker_dic)==3:
        for k,v in ex_group.items():
            
            start = v[1][0]
            for ux in v[1][1:len(v[1])]:
                if start in speaker_dic[0].keys():
                    if (ux in speaker_dic[1].keys()) or (ux in speaker_dic[2].keys()):
                        if (ux<u_high) and (v[0] <= u_high):
                            ex_in_window[k] = v
                            break
                elif start in speaker_dic[1].keys():
                    if (ux in speaker_dic[0].keys()) or (ux in speaker_dic[2].keys()):
                        if (ux<u_high) and (v[0] <= u_high):
                            ex_in_window[k] = v
                            break
                elif start in speaker_dic[2].keys():
                    if (ux in speaker_dic[0].keys()) or (ux in speaker_dic[1].keys()):
                        if (ux<u_high) and (v[0] <= u_high):
                            ex_in_window[k] = v
                            break
                        
    if len(speaker_dic)==4:
        for k,v in ex_group.items():
            flag = True
            start = v[1][0]
            for ux in v[1][1:len(v[1])]:
                if start in speaker_dic[0].keys():
                    if (ux in speaker_dic[1].keys()) or (ux in speaker_dic[2].keys()) or (ux in speaker_dic[3].keys()):
                        if (ux<u_high) and (v[0] <= u_high):
                            ex_in_window[k] = v
                            break
                elif start in speaker_dic[1].keys():
                    if (ux in speaker_dic[0].keys()) or (ux in speaker_dic[2].keys()) or (ux in speaker_dic[3].keys()):
                        if (ux<u_high) and (v[0] <= u_high):
                            ex_in_window[k] = v
                            break
                elif start in speaker_dic[2].keys():
                    if (ux in speaker_dic[0].keys()) or (ux in speaker_dic[1].keys()) or (ux in speaker_dic[3].keys()):
                        if (ux<u_high) and (v[0] <= u_high):
                            ex_in_window[k] = v
                            break
                elif start in speaker_dic[3].keys():
                    if (ux in speaker_dic[0].keys()) or (ux in speaker_dic[1].keys()) or (ux in speaker_dic[2].keys()):
                        if (ux<u_high) and (v[0] <= u_high):
                            ex_in_window[k] = v
                            break
                        
    return ex_in_window

In [142]:
def get_er_ratio_in_time_n(speaker, speaker_dic, ex_group, u_low, u_high):
    ex_in_window = get_er_window_n(speaker_dic, ex_group, u_high)
#     print(ex_in_window.keys())
    n = len(speaker)
    sum_token = [0 for i in range(0,n)]
    max_len = 0
    speaker_dic_string = [{} for i in range(0,n)]
    sum_er = [0 for i in range(0,n)]
    sum_er_token = [0 for i in range(0,n)]
    er_ratio = [0 for i in range(0,n)]
#     print(speaker_dic[2])
    
    for key,value in ex_in_window.items():
        max_len = max(max_len, len(key.split()))
    
    for i in range(0,n):
        for k,v in speaker_dic[i].items():
            if k<u_high and k>=u_low:
                sum_token[i] = sum_token[i]+len(speaker_dic[i][k])
                speaker_dic_string[i][k] = ' '.join(speaker_dic[i][k])
    
    m = max_len
    
    while m>1:
        for k,v in ex_in_window.items():
            if (len(k.split())==m):
                for u in ex_in_window[k][1]:
                    if u<u_high and u>= u_low:
                        for j in range(0,len(speaker_dic_string)):
                            if u in speaker_dic_string[j].keys():
                                c = speaker_dic_string[j][u].count(k)
                                speaker_dic_string[j][u] = speaker_dic_string[j][u].replace(k, '0'*len(k))
                                if u>= ex_in_window[k][0]:
                                    sum_er[j] = sum_er[j]+ c
                                    sum_er_token[j] = sum_er_token[j] + c*len(k.split())
        m = m-1
        
    for k,v in ex_in_window.items():
        if (len(k.split())==1):
            for u in ex_in_window[k][1]:
                if u<u_high and u>= u_low:
                    for j in range(0,len(speaker_dic_string)):
                        if u in speaker_dic_string[j].keys():
                            c = speaker_dic_string[j][u].split().count(k)
                            if u>= ex_in_window[k][0]:
                                sum_er[j] = sum_er[j]+ c
                                sum_er_token[j] = sum_er_token[j]+c
                            
    for c in range(0,len(sum_token)):
        if sum_token[c]!=0:
            er_ratio[c] = sum_er_token[c]/sum_token[c]
        else:
            er_ratio[c] = 0
    
    total_token_number = 0
    for tn in sum_token:
        total_token_number = total_token_number + tn
    
    total_er_token_number = 0
    for en in sum_er_token:
        total_er_token_number = total_er_token_number + en
        
    total_er_number = 0
    for er_n in sum_er:
        total_er_number = total_er_number + er_n
    
    if total_token_number!=0:
        av_er_ratio = total_er_token_number/total_token_number
    else:
        av_er_ratio = 0
    
    if n==3:
        result_dic = {'speaker1': speaker[0], 'speaker1_token_number': sum_token[0], 'speaker1_expression_token_number':sum_er_token[0], 
                      'speaker1_expression_number':sum_er[0],'speaker1_er_ratio':er_ratio[0],
                      'speaker2': speaker[1], 'speaker2_token_number': sum_token[1], 'speaker2_expression_token_number':sum_er_token[1], 
                      'speaker2_expression_number':sum_er[1],'speaker2_er_ratio':er_ratio[1],
                      'speaker3': speaker[2], 'speaker3_token_number': sum_token[2], 'speaker3_expression_token_number':sum_er_token[2], 
                      'speaker3_expression_number':sum_er[2],'speaker3_er_ratio':er_ratio[2],
                      'total_token_number': total_token_number, 'total_er_token_number' : total_er_token_number, 'av_er_ratio':av_er_ratio,
                      'total_er_number': total_er_number}
    if n==4:
        result_dic = {'speaker1': speaker[0], 'speaker1_token_number': sum_token[0], 'speaker1_expression_token_number':sum_er_token[0], 
                      'speaker1_expression_number':sum_er[0],'speaker1_er_ratio':er_ratio[0],
                      'speaker2': speaker[1], 'speaker2_token_number': sum_token[1], 'speaker2_expression_token_number':sum_er_token[1], 
                      'speaker2_expression_number':sum_er[1],'speaker2_er_ratio':er_ratio[1],
                      'speaker3': speaker[2], 'speaker3_token_number': sum_token[2], 'speaker3_expression_token_number':sum_er_token[2], 
                      'speaker3_expression_number':sum_er[2],'speaker3_er_ratio':er_ratio[2],
                      'speaker4': speaker[3], 'speaker4_token_number': sum_token[3], 'speaker4_expression_token_number':sum_er_token[3], 
                      'speaker4_expression_number':sum_er[3],'speaker4_er_ratio':er_ratio[3],
                      'total_token_number': total_token_number, 'total_er_token_number' : total_er_token_number, 'av_er_ratio':av_er_ratio,
                      'total_er_number': total_er_number}
        
    
    return result_dic
    
        
        

In [143]:
def get_er_ratio_in_time_n_o(speaker, speaker_dic, ex_group, u_low, u_high):
    ex_in_window = get_er_window_n(speaker_dic, ex_group, u_high)
#     print(ex_in_window.keys())
    n = len(speaker)
    sum_token = [0 for i in range(0,n)]
    max_len = 0
    speaker_dic_string = [{} for i in range(0,n)]
    sum_er = [0 for i in range(0,n)]
    sum_er_token = [0 for i in range(0,n)]
    er_ratio = [0 for i in range(0,n)]
    result_dic = {}
#     print(speaker_dic[2])
    
    for key,value in ex_in_window.items():
        max_len = max(max_len, len(key.split()))
    
    for i in range(0,n):
        for k,v in speaker_dic[i].items():
            if k<u_high and k>=u_low:
                sum_token[i] = sum_token[i]+len(speaker_dic[i][k])
                speaker_dic_string[i][k] = ' '.join(speaker_dic[i][k])
    
    m = max_len
    
    while m>0:
        for k,v in ex_in_window.items():
            if (len(k.split())==m):
                for u in ex_in_window[k]:
                    if u<u_high and u>= u_low:
                        for j in range(0,len(speaker_dic_string)):
                            if u in speaker_dic_string[j].keys():
                                sum_er[j] = sum_er[j]+ speaker_dic_string[j][u].count(' '+k)
                                sum_er_token[j] = sum_er_token[j]+speaker_dic_string[j][u].count(' '+k)*m
                                speaker_dic_string[j][u] = speaker_dic_string[j][u].replace(k, '0'*len(k))
#                                 print(speaker_dic_string[j][u],sum_er_token[j])
        m = m-1
        
#     for k,v in ex_in_window.items():
#         if (len(k.split())==1):
#             for u in ex_in_window[k]:
#                 if u<u_high and u>= u_low:
#                     for j in range(0,len(speaker_dic_string)):
#                         if u in speaker_dic_string[j].keys(): 
#                             sum_er[j] = sum_er[j]+ speaker_dic_string[j][u].split().count(k)
#                             sum_er_token[j] = sum_er_token[j]+speaker_dic_string[j][u].split().count(k)
                            
    for c in range(0,len(sum_token)):
        if sum_token[c]!=0:
            er_ratio[c] = sum_er_token[c]/sum_token[c]
        else:
            er_ratio[c] = 0
    
    total_token_number = 0
    for tn in sum_token:
        total_token_number = total_token_number + tn
    
    total_er_token_number = 0
    for en in sum_er_token:
        total_er_token_number = total_er_token_number + en
        
    total_er_number = 0
    for er_n in sum_er_token:
        total_er_number = total_er_number + er_n
    
    if total_token_number!=0:
        av_er_ratio = total_er_token_number/total_token_number
    else:
        av_er_ratio = 0
    
    if n==3:
        result_dic = {'speaker1': speaker[0], 'speaker1_token_number': sum_token[0], 'speaker1_expression_token_number':sum_er_token[0], 
                      'speaker1_expression_number':sum_er[0],'speaker1_er_ratio':er_ratio[0],
                      'speaker2': speaker[1], 'speaker2_token_number': sum_token[1], 'speaker2_expression_token_number':sum_er_token[1], 
                      'speaker2_expression_number':sum_er[1],'speaker2_er_ratio':er_ratio[1],
                      'speaker3': speaker[2], 'speaker3_token_number': sum_token[2], 'speaker3_expression_token_number':sum_er_token[2], 
                      'speaker3_expression_number':sum_er[2],'speaker3_er_ratio':er_ratio[2],
                      'total_token_number': total_token_number, 'total_er_token_number' : total_er_token_number, 'av_er_ratio':av_er_ratio,
                      'total_er_number': total_er_number}
    if n==4:
        result_dic = {'speaker1': speaker[0], 'speaker1_token_number': sum_token[0], 'speaker1_expression_token_number':sum_er_token[0], 
                      'speaker1_expression_number':sum_er[0],'speaker1_er_ratio':er_ratio[0],
                      'speaker2': speaker[1], 'speaker2_token_number': sum_token[1], 'speaker2_expression_token_number':sum_er_token[1], 
                      'speaker2_expression_number':sum_er[1],'speaker2_er_ratio':er_ratio[1],
                      'speaker3': speaker[2], 'speaker3_token_number': sum_token[2], 'speaker3_expression_token_number':sum_er_token[2], 
                      'speaker3_expression_number':sum_er[2],'speaker3_er_ratio':er_ratio[2],
                      'speaker4': speaker[3], 'speaker4_token_number': sum_token[3], 'speaker4_expression_token_number':sum_er_token[3], 
                      'speaker4_expression_number':sum_er[3],'speaker4_er_ratio':er_ratio[3],
                      'total_token_number': total_token_number, 'total_er_token_number' : total_er_token_number, 'av_er_ratio':av_er_ratio,
                      'total_er_number': total_er_number}
        
                     
    
    
    return result_dic

In [144]:
def number_of_slot_n(speaker_dic,length_u):
    number_of_u = 0
    for s_d in speaker_dic:
        number_of_u = number_of_u + len(s_d)
    
    number_slot = int(number_of_u - number_of_u%length_u)/length_u
    return int(number_slot)

In [145]:
def number_of_total_u(speaker_dic):
    number_of_u = 0
    for s_d in speaker_dic:
        number_of_u = number_of_u + len(s_d)
    
    return number_of_u

In [146]:
def get_er_ratio_temporal_n(speaker, speaker_dic, ex_group, length):
    slot_number = number_of_slot_n(speaker_dic, length)
    er_dic_list = []
    total_u = 0
    
    for s in speaker_dic:
        total_u = total_u+len(s)
    
    for i in range(0,slot_number):
        er_dic = get_er_ratio_in_time_n(speaker,speaker_dic, ex_group , length*i, length*(i+1))
        er_dic['begin'] = length*i
        er_dic['end'] = length*(i+1)
        er_dic_list.append(er_dic)
    
    er_dic_end = get_er_ratio_in_time_n(speaker, speaker_dic, ex_group , slot_number*length, total_u)
    er_dic_end['begin'] = slot_number*length
    er_dic_end['end'] = total_u
    er_dic_list.append(er_dic_end)
    
    return er_dic_list

In [147]:
out1 = extract_speakers_n(df,4)
print(out1)

(DataFrame[_c0: string, _c1: string], ['S0653', 'S0655', 'S0654', 'S0652'])


In [148]:
out2 = extract_dialogues_dic_n(out1[0],out1[1])

In [149]:
print(out2[0])

['S0653', 'S0655', 'S0654', 'S0652']


In [150]:
out3 = get_er_dic_group(out2[0], out2[1])

In [151]:
print(len(out3))

270


In [152]:
for k,v in out3.items():
    print(k,',',v)

dream , [6, [0, 6, 11, 13, 25, 41, 46, 55, 56, 57, 170, 188]]
about , [26, [0, 26, 188]]
mm , [27, [2, 8, 15, 27, 36, 38, 40, 59, 61, 79, 81, 88, 89, 92, 114, 118, 120, 124, 130, 133, 151, 153, 155, 164, 170, 171, 176, 189, 191, 196, 205, 209, 226]]
a , [13, [1, 3, 13, 16, 17, 19, 25, 26, 31, 37, 39, 46, 49, 50, 58, 70, 75, 77, 93, 100, 108, 112, 113, 117, 122, 124, 125, 134, 170, 172, 178, 190, 192, 197, 206, 208, 228]]
be , [13, [3, 4, 7, 11, 13, 16, 17, 19, 25, 26, 33, 37, 39, 47, 48, 49, 51, 53, 54, 56, 58, 60, 66, 69, 75, 76, 77, 80, 85, 87, 90, 91, 94, 95, 96, 98, 100, 102, 113, 119, 132, 134, 144, 154, 156, 159, 163, 167, 169, 188, 190, 192, 193, 197, 203, 204, 206, 208, 210, 215, 217, 219, 229]]
last night , [14, [13, 14, 46]]
night , [14, [13, 14, 46]]
have , [16, [13, 16, 26, 30, 46, 49, 56, 93, 111, 112, 119, 123, 124, 125, 173, 174, 182, 183, 184, 186, 192, 193, 197, 200, 206, 208]]
a dream , [25, [13, 25, 46]]
or , [26, [13, 26, 199]]
be this , [37, [13, 37, 49, 132]]
you 

In [153]:
out4 = get_er_ratio_in_time_n(out2[0], out2[1], out3, 0, number_of_total_u(out2[1]))
print(out4)

{'speaker1': 'S0653', 'speaker1_token_number': 639, 'speaker1_expression_token_number': 426, 'speaker1_expression_number': 349, 'speaker1_er_ratio': 0.6666666666666666, 'speaker2': 'S0655', 'speaker2_token_number': 391, 'speaker2_expression_token_number': 53, 'speaker2_expression_number': 44, 'speaker2_er_ratio': 0.13554987212276215, 'speaker3': 'S0654', 'speaker3_token_number': 504, 'speaker3_expression_token_number': 233, 'speaker3_expression_number': 159, 'speaker3_er_ratio': 0.4623015873015873, 'speaker4': 'S0652', 'speaker4_token_number': 309, 'speaker4_expression_token_number': 163, 'speaker4_expression_number': 118, 'speaker4_er_ratio': 0.5275080906148867, 'total_token_number': 1843, 'total_er_token_number': 875, 'av_er_ratio': 0.4747693977211069, 'total_er_number': 670}


In [154]:
out5 = number_of_slot_n(out2[1], 50)

In [155]:
out6 = get_er_ratio_temporal_n(out2[0], out2[1], out3, 50)

In [156]:
for o in out6:
    print(o['av_er_ratio'])

0.18199233716475097
0.5969696969696969
0.625
0.6609336609336609
0.7169117647058824


In [25]:
import pandas as pd
path = '~/corpus/bnc_untagged_dialign/tsvfiles-4/output_er_summary_4_1_dialign_ex_sw'
path2 = '~/corpus/bnc_untagged_dialign/tsvfiles-4/output_er_summary_4_1_dialign_lexicon_ex_sw'
dl = []
dl_result = []
ex_general_summary = []
import glob,os
files = glob.glob('S***.tsv')
for f in files:
    dff = spark.read.csv(f, sep=r'\t', header=False, mode="DROPMALFORMED")
    speaker_f = extract_speakers_n(dff,4)
    dialogue_dic_f = extract_dialogues_dic_n(speaker_f[0],speaker_f[1])
    ex_dic_f = get_er_dic_group(dialogue_dic_f[0],dialogue_dic_f[1])
#     for k,v in ex_dic_f.items():
#         print(f,k,',',v)
#     print(dialogue_dic_f[1][4])
    er_result_f = get_er_ratio_temporal_n(dialogue_dic_f[0],dialogue_dic_f[1], ex_dic_f, 50)
    for fx in er_result_f:
        fx['file'] = f
    
    er_result_f_2 = get_er_ratio_in_time_n(dialogue_dic_f[0],dialogue_dic_f[1], ex_dic_f, 0, number_of_total_u(dialogue_dic_f[1]))
    er_result_f_2['file'] = f
    print(er_result_f_2)
    dl_result.append(er_result_f_2)
    
    dl_s = pd.DataFrame(er_result_f)
    dl_s.to_csv(os.path.join(path,f[0:5] + 'csv'), index = False)
    
    ex_for_each_file = []
    for ex, value in ex_dic_f.items():
        ex_for_each_file.append({'Expression':ex, 'Expression_length': len(ex.split()), 'Expression_established_position':value[0], 'Expression_uttered_position_speaker':value[1],'file':f})
    
    ex_general_summary.append({'Expression Lexicon Size without stopwords': len(ex_dic_f),'file':f})
    print({'Expression Lexicon Size without stopwords': len(ex_dic_f),'file':f})
    dl_ex_for_each_file = pd.DataFrame(ex_for_each_file)
    dl_ex_for_each_file.to_csv(os.path.join(path2,f[0:5] + 'csv'), index = False)
    

dl_r = pd.DataFrame(dl_result)
dl_r.to_csv(os.path.join(path,'general_result_4_1.csv'), index = False)
dl_ex_general_summary = pd.DataFrame(ex_general_summary)
dl_ex_general_summary.to_csv(os.path.join(path2,'expression_lexicon_size_4_1_ex_sw.csv'), index = False)

{'speaker1': 'S0653', 'speaker1_token_number': 639, 'speaker1_expression_token_number': 223, 'speaker1_expression_number': 146, 'speaker1_er_ratio': 0.3489827856025039, 'speaker2': 'S0655', 'speaker2_token_number': 391, 'speaker2_expression_token_number': 29, 'speaker2_expression_number': 20, 'speaker2_er_ratio': 0.0741687979539642, 'speaker3': 'S0654', 'speaker3_token_number': 504, 'speaker3_expression_token_number': 149, 'speaker3_expression_number': 75, 'speaker3_er_ratio': 0.29563492063492064, 'speaker4': 'S0652', 'speaker4_token_number': 309, 'speaker4_expression_token_number': 98, 'speaker4_expression_number': 53, 'speaker4_er_ratio': 0.31715210355987056, 'total_token_number': 1843, 'total_er_token_number': 499, 'av_er_ratio': 0.27075420510037984, 'total_er_number': 294, 'file': 'SN2C.tsv'}
{'Expression Lexicon Size without stopwords': 205, 'file': 'SN2C.tsv'}
{'speaker1': 'S0554', 'speaker1_token_number': 1337, 'speaker1_expression_token_number': 589, 'speaker1_expression_number

{'speaker1': 'S0003', 'speaker1_token_number': 1704, 'speaker1_expression_token_number': 767, 'speaker1_expression_number': 491, 'speaker1_er_ratio': 0.45011737089201875, 'speaker2': 'S0130', 'speaker2_token_number': 1769, 'speaker2_expression_token_number': 838, 'speaker2_expression_number': 523, 'speaker2_er_ratio': 0.47371396269078575, 'speaker3': 'S0113', 'speaker3_token_number': 678, 'speaker3_expression_token_number': 331, 'speaker3_expression_number': 206, 'speaker3_er_ratio': 0.4882005899705015, 'speaker4': 'S0159', 'speaker4_token_number': 449, 'speaker4_expression_token_number': 222, 'speaker4_expression_number': 138, 'speaker4_er_ratio': 0.49443207126948774, 'total_token_number': 4600, 'total_er_token_number': 2158, 'av_er_ratio': 0.4691304347826087, 'total_er_number': 1358, 'file': 'S7DT.tsv'}
{'Expression Lexicon Size without stopwords': 594, 'file': 'S7DT.tsv'}
{'speaker1': 'S0391', 'speaker1_token_number': 1440, 'speaker1_expression_token_number': 678, 'speaker1_expressi

{'speaker1': 'S0144', 'speaker1_token_number': 576, 'speaker1_expression_token_number': 215, 'speaker1_expression_number': 146, 'speaker1_er_ratio': 0.3732638888888889, 'speaker2': 'S0024', 'speaker2_token_number': 1404, 'speaker2_expression_token_number': 484, 'speaker2_expression_number': 315, 'speaker2_er_ratio': 0.34472934472934474, 'speaker3': 'S0012', 'speaker3_token_number': 1430, 'speaker3_expression_token_number': 608, 'speaker3_expression_number': 387, 'speaker3_er_ratio': 0.4251748251748252, 'speaker4': 'S0013', 'speaker4_token_number': 389, 'speaker4_expression_token_number': 138, 'speaker4_expression_number': 86, 'speaker4_er_ratio': 0.35475578406169667, 'total_token_number': 3799, 'total_er_token_number': 1445, 'av_er_ratio': 0.38036325348775996, 'total_er_number': 934, 'file': 'SQ2D.tsv'}
{'Expression Lexicon Size without stopwords': 482, 'file': 'SQ2D.tsv'}
{'speaker1': 'S0530', 'speaker1_token_number': 1741, 'speaker1_expression_token_number': 697, 'speaker1_expression

{'speaker1': 'S0192', 'speaker1_token_number': 21515, 'speaker1_expression_token_number': 12667, 'speaker1_expression_number': 7561, 'speaker1_er_ratio': 0.5887520334650244, 'speaker2': 'S0227', 'speaker2_token_number': 9248, 'speaker2_expression_token_number': 5087, 'speaker2_expression_number': 2912, 'speaker2_er_ratio': 0.5500648788927336, 'speaker3': 'S0190', 'speaker3_token_number': 3737, 'speaker3_expression_token_number': 2530, 'speaker3_expression_number': 1343, 'speaker3_er_ratio': 0.677013647310677, 'speaker4': 'S0189', 'speaker4_token_number': 3895, 'speaker4_expression_token_number': 2299, 'speaker4_expression_number': 1266, 'speaker4_er_ratio': 0.5902439024390244, 'total_token_number': 38395, 'total_er_token_number': 22583, 'av_er_ratio': 0.5881755436905847, 'total_er_number': 13082, 'file': 'SAUR.tsv'}
{'Expression Lexicon Size without stopwords': 4554, 'file': 'SAUR.tsv'}
{'speaker1': 'S0202', 'speaker1_token_number': 1077, 'speaker1_expression_token_number': 570, 'speak

{'speaker1': 'S0590', 'speaker1_token_number': 7450, 'speaker1_expression_token_number': 4237, 'speaker1_expression_number': 2630, 'speaker1_er_ratio': 0.5687248322147651, 'speaker2': 'S0591', 'speaker2_token_number': 1659, 'speaker2_expression_token_number': 778, 'speaker2_expression_number': 489, 'speaker2_er_ratio': 0.4689572031344183, 'speaker3': 'S0589', 'speaker3_token_number': 6251, 'speaker3_expression_token_number': 2858, 'speaker3_expression_number': 1843, 'speaker3_er_ratio': 0.45720684690449526, 'speaker4': 'S0588', 'speaker4_token_number': 4107, 'speaker4_expression_token_number': 2189, 'speaker4_expression_number': 1380, 'speaker4_er_ratio': 0.5329924519113708, 'total_token_number': 19467, 'total_er_token_number': 10062, 'av_er_ratio': 0.5168747110494684, 'total_er_number': 6342, 'file': 'SVBB.tsv'}
{'Expression Lexicon Size without stopwords': 2560, 'file': 'SVBB.tsv'}
{'speaker1': 'S0415', 'speaker1_token_number': 124, 'speaker1_expression_token_number': 49, 'speaker1_e

...

{'speaker1': 'S0337', 'speaker1_token_number': 1981, 'speaker1_expression_token_number': 910, 'speaker1_expression_number': 587, 'speaker1_er_ratio': 0.45936395759717313, 'speaker2': 'S0346', 'speaker2_token_number': 1979, 'speaker2_expression_token_number': 941, 'speaker2_expression_number': 612, 'speaker2_er_ratio': 0.47549267306720566, 'speaker3': 'S0336', 'speaker3_token_number': 2865, 'speaker3_expression_token_number': 1372, 'speaker3_expression_number': 867, 'speaker3_er_ratio': 0.4788830715532286, 'speaker4': 'S0347', 'speaker4_token_number': 1821, 'speaker4_expression_token_number': 840, 'speaker4_expression_number': 519, 'speaker4_er_ratio': 0.4612850082372323, 'total_token_number': 8646, 'total_er_token_number': 4063, 'av_er_ratio': 0.46992829053897756, 'total_er_number': 2585, 'file': 'SM99.tsv'}
{'Expression Lexicon Size without stopwords': 1172, 'file': 'SM99.tsv'}
{'speaker1': 'S0336', 'speaker1_token_number': 4790, 'speaker1_expression_token_number': 2596, 'speaker1_exp

{'speaker1': 'S0058', 'speaker1_token_number': 2689, 'speaker1_expression_token_number': 1347, 'speaker1_expression_number': 909, 'speaker1_er_ratio': 0.5009297136481964, 'speaker2': 'S0026', 'speaker2_token_number': 2594, 'speaker2_expression_token_number': 978, 'speaker2_expression_number': 650, 'speaker2_er_ratio': 0.37702390131071706, 'speaker3': 'S0120', 'speaker3_token_number': 2108, 'speaker3_expression_token_number': 896, 'speaker3_expression_number': 602, 'speaker3_er_ratio': 0.4250474383301708, 'speaker4': 'S0179', 'speaker4_token_number': 639, 'speaker4_expression_token_number': 290, 'speaker4_expression_number': 190, 'speaker4_er_ratio': 0.4538341158059468, 'total_token_number': 8030, 'total_er_token_number': 3511, 'av_er_ratio': 0.4372353673723537, 'total_er_number': 2351, 'file': 'SUKT.tsv'}
{'Expression Lexicon Size without stopwords': 1039, 'file': 'SUKT.tsv'}
{'speaker1': 'S0142', 'speaker1_token_number': 5341, 'speaker1_expression_token_number': 2852, 'speaker1_expres

{'speaker1': 'S0493', 'speaker1_token_number': 4268, 'speaker1_expression_token_number': 1948, 'speaker1_expression_number': 1282, 'speaker1_er_ratio': 0.4564198687910028, 'speaker2': 'S0492', 'speaker2_token_number': 2878, 'speaker2_expression_token_number': 1537, 'speaker2_expression_number': 983, 'speaker2_er_ratio': 0.534051424600417, 'speaker3': 'S0666', 'speaker3_token_number': 429, 'speaker3_expression_token_number': 155, 'speaker3_expression_number': 110, 'speaker3_er_ratio': 0.3613053613053613, 'speaker4': 'S0604', 'speaker4_token_number': 2402, 'speaker4_expression_token_number': 1237, 'speaker4_expression_number': 803, 'speaker4_er_ratio': 0.5149875104079933, 'total_token_number': 9977, 'total_er_token_number': 4877, 'av_er_ratio': 0.48882429588052523, 'total_er_number': 3178, 'file': 'SWGV.tsv'}
{'Expression Lexicon Size without stopwords': 1250, 'file': 'SWGV.tsv'}
{'speaker1': 'S0547', 'speaker1_token_number': 536, 'speaker1_expression_token_number': 202, 'speaker1_expres

{'speaker1': 'S0144', 'speaker1_token_number': 873, 'speaker1_expression_token_number': 403, 'speaker1_expression_number': 284, 'speaker1_er_ratio': 0.4616265750286369, 'speaker2': 'S0013', 'speaker2_token_number': 1196, 'speaker2_expression_token_number': 554, 'speaker2_expression_number': 365, 'speaker2_er_ratio': 0.46321070234113715, 'speaker3': 'S0012', 'speaker3_token_number': 2776, 'speaker3_expression_token_number': 1151, 'speaker3_expression_number': 718, 'speaker3_er_ratio': 0.4146253602305475, 'speaker4': 'S0024', 'speaker4_token_number': 1706, 'speaker4_expression_token_number': 742, 'speaker4_expression_number': 481, 'speaker4_er_ratio': 0.4349355216881594, 'total_token_number': 6551, 'total_er_token_number': 2850, 'av_er_ratio': 0.43504808426194475, 'total_er_number': 1848, 'file': 'SCZV.tsv'}
{'Expression Lexicon Size without stopwords': 828, 'file': 'SCZV.tsv'}
{'speaker1': 'S0339', 'speaker1_token_number': 1112, 'speaker1_expression_token_number': 457, 'speaker1_express

{'speaker1': 'S0411', 'speaker1_token_number': 1715, 'speaker1_expression_token_number': 701, 'speaker1_expression_number': 464, 'speaker1_er_ratio': 0.4087463556851312, 'speaker2': 'S0417', 'speaker2_token_number': 1391, 'speaker2_expression_token_number': 545, 'speaker2_expression_number': 355, 'speaker2_er_ratio': 0.391804457225018, 'speaker3': 'S0416', 'speaker3_token_number': 1469, 'speaker3_expression_token_number': 601, 'speaker3_expression_number': 406, 'speaker3_er_ratio': 0.4091218515997277, 'speaker4': 'S0412', 'speaker4_token_number': 139, 'speaker4_expression_token_number': 72, 'speaker4_expression_number': 40, 'speaker4_er_ratio': 0.5179856115107914, 'total_token_number': 4714, 'total_er_token_number': 1919, 'av_er_ratio': 0.40708527789563004, 'total_er_number': 1265, 'file': 'STA2.tsv'}
{'Expression Lexicon Size without stopwords': 584, 'file': 'STA2.tsv'}
{'speaker1': 'S0673', 'speaker1_token_number': 622, 'speaker1_expression_token_number': 284, 'speaker1_expression_nu

In [93]:
import pandas as pd
path = '~/corpus/bnc_untagged_dialign/tsvfiles-4/output_er_summary_4_1_dialign'
dl = []
dl_result = []
import glob,os
files = glob.glob('S***.tsv')
for f in files:
    dff = spark.read.csv(f, sep=r'\t', header=False, mode="DROPMALFORMED")
    speaker_f = extract_speakers_n(dff,4)
    dialogue_dic_f = extract_dialogues_dic_n(speaker_f[0],speaker_f[1])
    ex_dic_f = get_er_dic_group(dialogue_dic_f[0],dialogue_dic_f[1])
#     for k,v in ex_dic_f.items():
#         print(f,k,',',v)
#     print(dialogue_dic_f[1][4])
    er_result_f = get_er_ratio_temporal_n(dialogue_dic_f[0],dialogue_dic_f[1], ex_dic_f, 50)
    for fx in er_result_f:
        fx['file'] = f
    
    er_result_f_2 = get_er_ratio_in_time_n(dialogue_dic_f[0],dialogue_dic_f[1], ex_dic_f, 0, number_of_total_u(dialogue_dic_f[1]))
    er_result_f_2['file'] = f
    print(er_result_f_2)
    dl_result.append(er_result_f_2)
    
    dl_s = pd.DataFrame(er_result_f)
    dl_s.to_csv(os.path.join(path,f[0:5] + 'csv'), index = False)

dl_r = pd.DataFrame(dl_result)
dl_r.to_csv(os.path.join(path,'general_result_4_1.csv'), index = False)


{'speaker1': 'S0653', 'speaker1_token_number': 639, 'speaker1_expression_token_number': 426, 'speaker1_expression_number': 349, 'speaker1_er_ratio': 0.6666666666666666, 'speaker2': 'S0655', 'speaker2_token_number': 391, 'speaker2_expression_token_number': 53, 'speaker2_expression_number': 44, 'speaker2_er_ratio': 0.13554987212276215, 'speaker3': 'S0654', 'speaker3_token_number': 504, 'speaker3_expression_token_number': 233, 'speaker3_expression_number': 159, 'speaker3_er_ratio': 0.4623015873015873, 'speaker4': 'S0652', 'speaker4_token_number': 309, 'speaker4_expression_token_number': 163, 'speaker4_expression_number': 118, 'speaker4_er_ratio': 0.5275080906148867, 'total_token_number': 1843, 'total_er_token_number': 875, 'av_er_ratio': 0.4747693977211069, 'total_er_number': 670, 'file': 'SN2C.tsv'}
{'speaker1': 'S0554', 'speaker1_token_number': 1337, 'speaker1_expression_token_number': 872, 'speaker1_expression_number': 653, 'speaker1_er_ratio': 0.6522064323111444, 'speaker2': 'S0405', 

{'speaker1': 'S0391', 'speaker1_token_number': 1440, 'speaker1_expression_token_number': 961, 'speaker1_expression_number': 700, 'speaker1_er_ratio': 0.6673611111111111, 'speaker2': 'S0393', 'speaker2_token_number': 1126, 'speaker2_expression_token_number': 647, 'speaker2_expression_number': 457, 'speaker2_er_ratio': 0.5746003552397868, 'speaker3': 'S0392', 'speaker3_token_number': 1819, 'speaker3_expression_token_number': 1029, 'speaker3_expression_number': 774, 'speaker3_er_ratio': 0.565695437053326, 'speaker4': 'S0397', 'speaker4_token_number': 1701, 'speaker4_expression_token_number': 1044, 'speaker4_expression_number': 770, 'speaker4_er_ratio': 0.6137566137566137, 'total_token_number': 6086, 'total_er_token_number': 3681, 'av_er_ratio': 0.6048307591192902, 'total_er_number': 2701, 'file': 'SNE5.tsv'}
{'speaker1': 'S0571', 'speaker1_token_number': 7892, 'speaker1_expression_token_number': 5253, 'speaker1_expression_number': 3619, 'speaker1_er_ratio': 0.6656107450582869, 'speaker2':

{'speaker1': 'S0193', 'speaker1_token_number': 6340, 'speaker1_expression_token_number': 3965, 'speaker1_expression_number': 2842, 'speaker1_er_ratio': 0.6253943217665615, 'speaker2': 'S0192', 'speaker2_token_number': 9786, 'speaker2_expression_token_number': 7355, 'speaker2_expression_number': 5113, 'speaker2_er_ratio': 0.7515838953607193, 'speaker3': 'S0199', 'speaker3_token_number': 3614, 'speaker3_expression_token_number': 2417, 'speaker3_expression_number': 1656, 'speaker3_er_ratio': 0.6687880464858882, 'speaker4': 'S0191', 'speaker4_token_number': 91, 'speaker4_expression_token_number': 56, 'speaker4_expression_number': 43, 'speaker4_er_ratio': 0.6153846153846154, 'total_token_number': 19831, 'total_er_token_number': 13793, 'av_er_ratio': 0.6955272048812465, 'total_er_number': 9654, 'file': 'SVZB.tsv'}
{'speaker1': 'S0251', 'speaker1_token_number': 2836, 'speaker1_expression_token_number': 1832, 'speaker1_expression_number': 1329, 'speaker1_er_ratio': 0.6459802538787024, 'speaker

{'speaker1': 'S0401', 'speaker1_token_number': 819, 'speaker1_expression_token_number': 561, 'speaker1_expression_number': 441, 'speaker1_er_ratio': 0.684981684981685, 'speaker2': 'S0261', 'speaker2_token_number': 1037, 'speaker2_expression_token_number': 684, 'speaker2_expression_number': 506, 'speaker2_er_ratio': 0.6595949855351977, 'speaker3': 'S0388', 'speaker3_token_number': 1486, 'speaker3_expression_token_number': 924, 'speaker3_expression_number': 682, 'speaker3_er_ratio': 0.6218034993270525, 'speaker4': 'S0262', 'speaker4_token_number': 2558, 'speaker4_expression_token_number': 1407, 'speaker4_expression_number': 1098, 'speaker4_er_ratio': 0.5500390930414386, 'total_token_number': 5900, 'total_er_token_number': 3576, 'av_er_ratio': 0.6061016949152542, 'total_er_number': 2727, 'file': 'SBQZ.tsv'}
{'speaker1': 'S0415', 'speaker1_token_number': 239, 'speaker1_expression_token_number': 121, 'speaker1_expression_number': 103, 'speaker1_er_ratio': 0.5062761506276151, 'speaker2': 'S0

{'speaker1': 'S0652', 'speaker1_token_number': 671, 'speaker1_expression_token_number': 404, 'speaker1_expression_number': 312, 'speaker1_er_ratio': 0.6020864381520119, 'speaker2': 'S0655', 'speaker2_token_number': 337, 'speaker2_expression_token_number': 172, 'speaker2_expression_number': 127, 'speaker2_er_ratio': 0.5103857566765578, 'speaker3': 'S0653', 'speaker3_token_number': 1050, 'speaker3_expression_token_number': 526, 'speaker3_expression_number': 413, 'speaker3_er_ratio': 0.5009523809523809, 'speaker4': 'S0654', 'speaker4_token_number': 216, 'speaker4_expression_token_number': 90, 'speaker4_expression_number': 60, 'speaker4_er_ratio': 0.4166666666666667, 'total_token_number': 2274, 'total_er_token_number': 1192, 'av_er_ratio': 0.5241864555848724, 'total_er_number': 912, 'file': 'SP45.tsv'}
{'speaker1': 'S0024', 'speaker1_token_number': 1996, 'speaker1_expression_token_number': 1257, 'speaker1_expression_number': 880, 'speaker1_er_ratio': 0.6297595190380761, 'speaker2': 'S0025'

...

{'speaker1': 'S0400', 'speaker1_token_number': 2301, 'speaker1_expression_token_number': 1338, 'speaker1_expression_number': 941, 'speaker1_er_ratio': 0.5814863102998696, 'speaker2': 'S0246', 'speaker2_token_number': 4503, 'speaker2_expression_token_number': 2922, 'speaker2_expression_number': 2073, 'speaker2_er_ratio': 0.6489007328447701, 'speaker3': 'S0245', 'speaker3_token_number': 1247, 'speaker3_expression_token_number': 775, 'speaker3_expression_number': 536, 'speaker3_er_ratio': 0.6214915797914996, 'speaker4': 'S0249', 'speaker4_token_number': 4388, 'speaker4_expression_token_number': 3214, 'speaker4_expression_number': 2197, 'speaker4_er_ratio': 0.7324521422060164, 'total_token_number': 12439, 'total_er_token_number': 8249, 'av_er_ratio': 0.6631562022670633, 'total_er_number': 5747, 'file': 'STXT.tsv'}
{'speaker1': 'S0690', 'speaker1_token_number': 723, 'speaker1_expression_token_number': 410, 'speaker1_expression_number': 303, 'speaker1_er_ratio': 0.5670816044260027, 'speaker2

{'speaker1': 'S0345', 'speaker1_token_number': 4069, 'speaker1_expression_token_number': 2909, 'speaker1_expression_number': 2016, 'speaker1_er_ratio': 0.7149176701892357, 'speaker2': 'S0344', 'speaker2_token_number': 5251, 'speaker2_expression_token_number': 3122, 'speaker2_expression_number': 2120, 'speaker2_er_ratio': 0.5945534183964959, 'speaker3': 'S0337', 'speaker3_token_number': 1880, 'speaker3_expression_token_number': 1218, 'speaker3_expression_number': 878, 'speaker3_er_ratio': 0.6478723404255319, 'speaker4': 'S0336', 'speaker4_token_number': 4539, 'speaker4_expression_token_number': 3203, 'speaker4_expression_number': 2168, 'speaker4_er_ratio': 0.7056620400969377, 'total_token_number': 15739, 'total_er_token_number': 10452, 'av_er_ratio': 0.6640828515153441, 'total_er_number': 7182, 'file': 'S85D.tsv'}
{'speaker1': 'S0094', 'speaker1_token_number': 1124, 'speaker1_expression_token_number': 759, 'speaker1_expression_number': 604, 'speaker1_er_ratio': 0.6752669039145908, 'spea

{'speaker1': 'S0273', 'speaker1_token_number': 2505, 'speaker1_expression_token_number': 1270, 'speaker1_expression_number': 979, 'speaker1_er_ratio': 0.5069860279441117, 'speaker2': 'S0266', 'speaker2_token_number': 1238, 'speaker2_expression_token_number': 923, 'speaker2_expression_number': 645, 'speaker2_er_ratio': 0.7455573505654282, 'speaker3': 'S0272', 'speaker3_token_number': 3931, 'speaker3_expression_token_number': 2813, 'speaker3_expression_number': 2101, 'speaker3_er_ratio': 0.7155939964385652, 'speaker4': 'S0268', 'speaker4_token_number': 44, 'speaker4_expression_token_number': 31, 'speaker4_expression_number': 24, 'speaker4_er_ratio': 0.7045454545454546, 'total_token_number': 7718, 'total_er_token_number': 5037, 'av_er_ratio': 0.6526302150816273, 'total_er_number': 3749, 'file': 'SZBD.tsv'}
{'speaker1': 'S0260', 'speaker1_token_number': 2096, 'speaker1_expression_token_number': 901, 'speaker1_expression_number': 711, 'speaker1_er_ratio': 0.42986641221374045, 'speaker2': 'S

{'speaker1': 'S0330', 'speaker1_token_number': 728, 'speaker1_expression_token_number': 485, 'speaker1_expression_number': 353, 'speaker1_er_ratio': 0.6662087912087912, 'speaker2': 'S0326', 'speaker2_token_number': 851, 'speaker2_expression_token_number': 530, 'speaker2_expression_number': 391, 'speaker2_er_ratio': 0.6227967097532315, 'speaker3': 'S0331', 'speaker3_token_number': 1516, 'speaker3_expression_token_number': 786, 'speaker3_expression_number': 593, 'speaker3_er_ratio': 0.5184696569920845, 'speaker4': 'S0328', 'speaker4_token_number': 1768, 'speaker4_expression_token_number': 1144, 'speaker4_expression_number': 852, 'speaker4_er_ratio': 0.6470588235294118, 'total_token_number': 4863, 'total_er_token_number': 2945, 'av_er_ratio': 0.6055932551922681, 'total_er_number': 2189, 'file': 'SWF6.tsv'}
{'speaker1': 'S0012', 'speaker1_token_number': 1036, 'speaker1_expression_token_number': 653, 'speaker1_expression_number': 517, 'speaker1_er_ratio': 0.6303088803088803, 'speaker2': 'S0

{'speaker1': 'S0361', 'speaker1_token_number': 4972, 'speaker1_expression_token_number': 3172, 'speaker1_expression_number': 2249, 'speaker1_er_ratio': 0.6379726468222043, 'speaker2': 'S0261', 'speaker2_token_number': 2443, 'speaker2_expression_token_number': 1710, 'speaker2_expression_number': 1223, 'speaker2_er_ratio': 0.6999590667212444, 'speaker3': 'S0262', 'speaker3_token_number': 3048, 'speaker3_expression_token_number': 2160, 'speaker3_expression_number': 1681, 'speaker3_er_ratio': 0.7086614173228346, 'speaker4': 'S0360', 'speaker4_token_number': 5555, 'speaker4_expression_token_number': 3768, 'speaker4_expression_number': 2782, 'speaker4_er_ratio': 0.6783078307830783, 'total_token_number': 16018, 'total_er_token_number': 10810, 'av_er_ratio': 0.6748657760019977, 'total_er_number': 7935, 'file': 'SYHP.tsv'}


In [157]:
#get expression lexicon
import pandas as pd
path = '~/corpus/bnc_untagged_dialign/tsvfiles-4/output_er_summary_4_1_dialign_lexicon'

import glob,os
files = glob.glob('S***.tsv')
ex_general_summary = []
for f in files:
    dff = spark.read.csv(f, sep=r'\t', header=False, mode="DROPMALFORMED")
    speaker_f = extract_speakers_n(dff,4)
    dialogue_dic_f = extract_dialogues_dic_n(speaker_f[0],speaker_f[1])
    ex_dic_f = get_er_dic_group(dialogue_dic_f[0],dialogue_dic_f[1])
    ex_for_each_file = []
    for ex, value in ex_dic_f.items():
        ex_for_each_file.append({'Expression':ex, 'Expression_length': len(ex.split()), 'Expression_established_position':value[0], 'Expression_uttered_position':value[1], 'file':f})
    
    ex_general_summary.append({'Expression Lexicon Size': len(ex_dic_f),'file':f})
    print({'Expression Lexicon Size': len(ex_dic_f),'file':f})
    
    dl_ex_for_each_file = pd.DataFrame(ex_for_each_file)
    dl_ex_for_each_file.to_csv(os.path.join(path,f[0:5] + 'csv'), index = False)
    
dl_ex_general_summary = pd.DataFrame(ex_general_summary)
dl_ex_general_summary.to_csv(os.path.join(path,'expression_lexicon_size_4_1.csv'), index = False)

{'Expression Lexicon Size': 270, 'file': 'SN2C.tsv'}
{'Expression Lexicon Size': 603, 'file': 'S29Q.tsv'}
{'Expression Lexicon Size': 1088, 'file': 'SNPA.tsv'}
{'Expression Lexicon Size': 1226, 'file': 'SAVN.tsv'}
{'Expression Lexicon Size': 1727, 'file': 'SCXG.tsv'}
{'Expression Lexicon Size': 730, 'file': 'SQD6.tsv'}
{'Expression Lexicon Size': 2505, 'file': 'SXLC.tsv'}
{'Expression Lexicon Size': 2538, 'file': 'SMZV.tsv'}
{'Expression Lexicon Size': 537, 'file': 'S9RV.tsv'}
{'Expression Lexicon Size': 2689, 'file': 'S7JH.tsv'}
{'Expression Lexicon Size': 680, 'file': 'S7DT.tsv'}
{'Expression Lexicon Size': 966, 'file': 'SNE5.tsv'}
{'Expression Lexicon Size': 2549, 'file': 'SN3D.tsv'}
{'Expression Lexicon Size': 522, 'file': 'S5SJ.tsv'}
{'Expression Lexicon Size': 726, 'file': 'SLPW.tsv'}
{'Expression Lexicon Size': 907, 'file': 'SWSL.tsv'}
{'Expression Lexicon Size': 622, 'file': 'SM6J.tsv'}
{'Expression Lexicon Size': 569, 'file': 'SAZY.tsv'}
{'Expression Lexicon Size': 913, 'file'

{'Expression Lexicon Size': 1892, 'file': 'STXT.tsv'}
{'Expression Lexicon Size': 1034, 'file': 'S8CV.tsv'}
{'Expression Lexicon Size': 419, 'file': 'S86J.tsv'}
{'Expression Lexicon Size': 1053, 'file': 'SVZH.tsv'}
{'Expression Lexicon Size': 1472, 'file': 'SZ95.tsv'}
{'Expression Lexicon Size': 744, 'file': 'S47C.tsv'}
{'Expression Lexicon Size': 628, 'file': 'SEBP.tsv'}
{'Expression Lexicon Size': 1143, 'file': 'SUKT.tsv'}
{'Expression Lexicon Size': 1577, 'file': 'SP6E.tsv'}
{'Expression Lexicon Size': 375, 'file': 'S477.tsv'}
{'Expression Lexicon Size': 1335, 'file': 'SVX4.tsv'}
{'Expression Lexicon Size': 2260, 'file': 'S85D.tsv'}
{'Expression Lexicon Size': 538, 'file': 'SHKF.tsv'}
{'Expression Lexicon Size': 1937, 'file': 'SJRQ.tsv'}
{'Expression Lexicon Size': 327, 'file': 'S8YL.tsv'}
{'Expression Lexicon Size': 668, 'file': 'SBKM.tsv'}
{'Expression Lexicon Size': 311, 'file': 'S5P8.tsv'}
{'Expression Lexicon Size': 1355, 'file': 'SWGV.tsv'}
{'Expression Lexicon Size': 358, 'fi